In [2]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
import IPython.display as ipd
from PIL import Image
import os
import scipy.signal as signal
from tqdm import tqdm
import random

In [3]:
# Step 1: Denoising using a low pass filter
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [4]:
# Downsampling audio
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [5]:
# Split audio into fixed-length segments
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [6]:
def norm_mfcc(audio_path):

    # Load audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Compute MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=48)

    # Normalize MFCC values
    normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
    return normalized_mfcc

In [7]:
def time_warp(mfcc, time_warping_range):
    num_frames = mfcc.shape[1]
    t = np.arange(num_frames)
    random_shift = np.random.randint(-time_warping_range, time_warping_range)
    warped_t = np.clip(t + random_shift, 0, num_frames - 1)
    warped_mfcc = mfcc[:, warped_t]
    return warped_mfcc[:, :48]

In [8]:
def frequency_mask(mfcc, frequency_masking_para, mask_max_frames):
    num_mfcc = mfcc.shape[0]
    num_frames = mfcc.shape[1]
    f = np.random.randint(0, frequency_masking_para)
    f0 = random.randint(0, num_mfcc - f)
    mfcc[f0:f0 + f, :] = 0.0
    return mfcc[:, :48]

In [9]:
Df_A=pd.read_csv(r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Dataframes\Heart_Audio_Default_Folders_A.csv")

In [10]:
Df_A

,Unnamed: 0,File,Class,Folder
0,0,201012172012.wav,Artifact,Atraining_artifact
1,1,201105040918.wav,Artifact,Atraining_artifact
2,2,201105041959.wav,Artifact,Atraining_artifact
3,3,201105051017.wav,Artifact,Atraining_artifact
4,4,201105060108.wav,Artifact,Atraining_artifact
...,...,...,...,...
119,119,201103200218.wav,Extrasystole,Atraining_extrahls
120,120,201104021355.wav,Extrasystole,Atraining_extrahls
121,121,201104140118.wav,Extrasystole,Atraining_extrahls
122,122,201104270458.wav,Extrasystole,Atraining_extrahls


In [11]:
input_shape = (39,32)  # Example input shape for MobileNetV2

In [12]:
MFCC_Dataset=[]
folder_path = r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Time_Shifted_Audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [13]:
len(MFCC_Dataset)

290

In [14]:
folder_path = r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Pitch_Shifted_Audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [15]:
len(MFCC_Dataset)

580

In [16]:
folder_path = r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Normalized_Audio"

# Get the file names from the folder
file_names = os.listdir(folder_path)

# Print the file names
for file_name in file_names:
    mfcc_class=Df_A.loc[Df_A["File"]==(str(file_name[:12])+".wav"),"Class"].iloc[0]
    audio_path = os.path.join(os.path.abspath(folder_path),file_name)
    mfcc_array=norm_mfcc(audio_path)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(mfcc_array, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    
    # Set time warping range (adjust according to your needs)
    time_warping_range = 5

    warped_mfcc = time_warp(mfcc_array, time_warping_range)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(warped_mfcc,input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    
    # Set frequency masking parameters (adjust according to your needs)
    frequency_masking_para = 5
    mask_max_frames = 5

    masked_mfcc = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
    masked_mfcc2 = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
    # Reshape MFCC array
    mfcc_reshaped = np.reshape(masked_mfcc, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])
    mfcc_reshaped = np.reshape(masked_mfcc2, input_shape)
    MFCC_Dataset.append([mfcc_reshaped,mfcc_class])

In [17]:
len(MFCC_Dataset)

1740

In [18]:
MFCC_Df=pd.DataFrame(MFCC_Dataset, columns=['MFCC', 'Class'])

In [19]:
MFCC_Df

,MFCC,Class
0,"[[-7.1183896, -7.124328, -7.135321, -6.925219,...",Artifact
1,"[[-7.2328024, -7.1927686, -7.1960464, -7.21481...",Artifact
2,"[[-7.127256, -6.878575, -6.8865304, -6.8151107...",Artifact
3,"[[-6.463061, -6.445672, -6.377971, -6.424771, ...",Murmur
4,"[[-5.1188455, -5.630413, -6.7886634, -6.723926...",Murmur
...,...,...
1735,"[[-6.6211963, -6.4577603, -6.398823, -6.444192...",Murmur
1736,"[[-6.3066964, -6.2473, -6.4322557, -6.5906267,...",Murmur
1737,"[[-6.5906267, -6.48989, -6.461418, -6.5676994,...",Murmur
1738,"[[-6.3066964, -6.2473, -6.4322557, -6.5906267,...",Murmur


In [20]:
MFCC_Df['Class'].value_counts()

Class
Artifact        720
Normal          408
Murmur          390
Extrasystole    222
Name: count, dtype: int64

In [37]:
for i in MFCC_Df["MFCC"]:
    print(i.shape)

(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(

In [38]:
# Define the column value to filter
column_value = 'Artifact'

# Filter the DataFrame based on the column value
filtered_df = MFCC_Df[MFCC_Df['Class'] == column_value]

# Calculate the number of rows to drop (half of the filtered rows)
rows_to_drop = len(filtered_df) // 2

# Randomly select rows to drop
rows = np.random.choice(filtered_df.index, rows_to_drop, replace=False)

# Drop the selected rows
MFCC_Final = MFCC_Df.drop(rows)

# Print the updated DataFrame
print(MFCC_Final)

                                                   MFCC     Class
1     [[-7.2328024, -7.1927686, -7.1960464, -7.21481...  Artifact
3     [[-6.463061, -6.445672, -6.377971, -6.424771, ...    Murmur
4     [[-5.1188455, -5.630413, -6.7886634, -6.723926...    Murmur
5     [[-6.5309367, -6.5444617, -6.552616, -6.467783...    Murmur
6     [[-6.6437407, -6.506994, -6.591128, -6.5763826...    Murmur
...                                                 ...       ...
1735  [[-6.6211963, -6.4577603, -6.398823, -6.444192...    Murmur
1736  [[-6.3066964, -6.2473, -6.4322557, -6.5906267,...    Murmur
1737  [[-6.5906267, -6.48989, -6.461418, -6.5676994,...    Murmur
1738  [[-6.3066964, -6.2473, -6.4322557, -6.5906267,...    Murmur
1739  [[-6.3066964, -6.2473, -6.4322557, -6.5906267,...    Murmur

[1380 rows x 2 columns]


In [39]:
MFCC_Final['Class'].value_counts()

Class
Normal          408
Murmur          390
Artifact        360
Extrasystole    222
Name: count, dtype: int64

In [40]:
X=np.array(MFCC_Final["MFCC"].tolist())
y=np.array(MFCC_Final["Class"].tolist())

In [41]:
X.shape

(1380, 39, 32)

In [42]:
input_shape = (39, 32, 3)
X = np.repeat(X[..., np.newaxis], 3, axis=-1)

In [43]:
np.unique(y)

array(['Artifact', 'Extrasystole', 'Murmur', 'Normal'], dtype='<U12')

In [44]:
y=np.array(pd.get_dummies(y))

In [45]:
y

array([[ True, False, False, False],
       [False, False,  True, False],
       [False, False,  True, False],
       ...,
       [False, False,  True, False],
       [False, False,  True, False],
       [False, False,  True, False]])

In [46]:
X.shape

(1380, 39, 32, 3)

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [48]:
X_train.shape

(1104, 39, 32, 3)

In [49]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [50]:
X_train.shape

(938, 39, 32, 3)

In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D,MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [52]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained ResNet152V2 model
resnet152v2 = ResNet152V2(weights='imagenet', include_top=False, input_shape=(39, 32, 3))

# Freeze the pre-trained layers
for layer in resnet152v2.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(resnet152v2.output)
dense_layer = Dense(128, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.35)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=resnet152v2.input, outputs=classification_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 39, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 45, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 20, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 22, 18, 64)   0           ['conv1_conv[0][0]']             
                                                                                            

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 10, 8, 64)    16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 10, 8, 64)   256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 10, 8, 64)   0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 5, 4, 128)   0           ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 5, 4, 512)    66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 5, 4, 512)    0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 5, 4, 128)   0           ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 7, 6, 128)   0           ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 5, 4, 128)    147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_block5_2_bn (BatchNormal  (None, 5, 4, 128)   512         ['conv3_block5_2_conv[0][0]']    
 ization) 

 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 5, 4, 512)   0           ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 5, 4, 128)    65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 5, 4, 128)   512         ['conv3_block8_1_conv[0][0]']    
 ization)                                                                                         
          

...

 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 3, 2, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 5, 4, 256)   0           ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 3, 2, 256)    589824      ['conv4_block29_2_pad[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block32_preact_bn (Batch  (None, 3, 2, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 3, 2, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 3, 2, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_block32_1_bn (BatchNorma  (None, 3, 2, 256)   1024        ['conv4_block32_1_conv[0][0]']   
 lization)

 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 3, 2, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 3, 2, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 3, 2, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
                                                                                                  
 conv4_blo

 conv5_block1_1_bn (BatchNormal  (None, 2, 1, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 2, 1, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 4, 3, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_block1_2_conv (Conv2D)   (None, 2, 1, 512)    2359296     ['conv5_block1_2_pad[0][0]']     
          

                                                                                                  
 conv5_block3_out (Add)         (None, 2, 1, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 2, 1, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 2, 1, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 flatten_1 (Flatten)            (None, 4096)         0           ['post_relu[0][0]']              
                                                                                                  
 dense_2 (

In [53]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
checkpoint = ModelCheckpoint('heart_resnet_mfcc.hdf5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[checkpoint])

Epoch 1/50
30/30 [==============================] - 53s 934ms/step - loss: 1.1638 - accuracy: 0.5064 - val_loss: 0.8222 - val_accuracy: 0.6807
Epoch 2/50
30/30 [==============================] - 23s 760ms/step - loss: 0.7602 - accuracy: 0.6823 - val_loss: 0.6747 - val_accuracy: 0.7108
Epoch 3/50
30/30 [==============================] - 21s 691ms/step - loss: 0.6854 - accuracy: 0.7367 - val_loss: 0.6207 - val_accuracy: 0.7590
Epoch 4/50
30/30 [==============================] - 18s 607ms/step - loss: 0.6261 - accuracy: 0.7527 - val_loss: 0.5887 - val_accuracy: 0.7530
Epoch 5/50
30/30 [==============================] - 21s 689ms/step - loss: 0.5536 - accuracy: 0.7740 - val_loss: 0.5581 - val_accuracy: 0.7711
Epoch 6/50
30/30 [==============================] - 23s 768ms/step - loss: 0.4986 - accuracy: 0.7964 - val_loss: 0.5266 - val_accuracy: 0.8193
Epoch 7/50
30/30 [==============================] - 18s 610ms/step - loss: 0.4612 - accuracy: 0.8188 - val_loss: 0.4813 - val_accuracy: 0.8012

In [55]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9384058117866516


In [62]:
def pred(audio_path, model_path):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)
    
    model = keras.models.load_model(model_path)
    i=0
    
    for segment in segments:
        # Compute MFCC features
        mfcc = librosa.feature.mfcc(y=segment, sr=target_sampling_rate, n_mfcc=48)

        # Normalize MFCC values
        normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        mfcc_reshaped = np.reshape(normalized_mfcc,(1,39,32))
        
        input_shape = (39, 32, 3)
        mfcc_final = np.repeat(mfcc_reshaped[..., np.newaxis], 3, axis=-1)
        
        prediction = model.predict(mfcc_final)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        i=i+1

#         prediction = model.predict(img_arr)
#         x=np.argmax(prediction)

        classes={0:'Artifact', 1:'Extrasystole', 2:'Murmur', 3:'Normal'}
        print(classes[x],confidence)
#         print(mfcc_final.shape)

In [172]:
audio_path=r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Atraining_extrahls\201101160808.wav"
model_path=r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Models\heart_mobilenet_mfcc_8.hdf5"
pred(audio_path,model_path)

1/1 [==============================] - 1s 706ms/step
Extrasystole 0.9999813
1/1 [==============================] - 0s 24ms/step
Extrasystole 0.99992406


In [68]:
audio_path=r"C:\Users\bharg\Downloads\mixkit-inside-a-beating-heart-496.wav"
model_path=r"C:\Users\bharg\Desktop\Heart_Sounds\DatasetA\Models\heart_mobilenet_mfcc_8_downsampled.hdf5"
pred(audio_path,model_path)

1/1 [==============================] - 1s 1s/step
Normal 0.997938
1/1 [==============================] - 0s 46ms/step
Murmur 0.99999857
1/1 [==============================] - 0s 52ms/step
Normal 0.9838365
1/1 [==============================] - 0s 41ms/step
Murmur 0.99289894
1/1 [==============================] - 0s 52ms/step
Normal 0.613056
1/1 [==============================] - 0s 46ms/step
Normal 0.9926103
